In [ ]:
from flask import Flask
from flask import jsonify
import pyodbc
from flask_cors import CORS
import random

app = Flask(__name__)
CORS(app)

connection = pyodbc.connect('Driver={SQL Server};'
                            'Server=localhost,1433;'
                            'Database=BEERTEST;'
                            'UID=SA;'
                            'PWD=uKmzTFy6f')
cursor = connection.cursor()


@app.route('/BEER_TOP/<num_to_return>/<direction>/<return_type>/<min_review_num>')
def return_beer_top(num_to_return, direction, return_type, min_review_num):
    cursor.execute(f'SELECT TOP {num_to_return} * FROM BEER WHERE review_num > {min_review_num} ORDER BY {return_type} {direction}')
    rows = cursor.fetchall()
    
    data_to_return = []
     
#     print(rows)
            
    for row in rows:
        
        data_to_return.append(
            {
            'beer_id': f"{row[0]}", 
            'review_num': f"{row[1]}",
            'overall_mean': f"{round(row[2],2)}",
            'aroma_mean': f"{round(row[3],3)}",
            'appearance_mean': f"{round(row[4],2)}",
            'palate_mean': f"{round(row[5],2)}",
            'taste_mean': f"{round(row[6],2)}",
            'overall_mean_lower': f"{round(row[7],2)}",
            'brewery_name': f"{row[8]}",
            'beer_name': f"{row[9]}",
            'beer_style': f"{row[10]}",
            'beer_abv': f"{round(row[11],2)}"
            }
        )
        
    return jsonify(data_to_return)


@app.route('/USER_TOP/<num_to_return>/<direction>/<return_type>/<min_review_num>')
def return_user_top(num_to_return, direction, return_type, min_review_num):
    cursor.execute(f'SELECT TOP {num_to_return} * FROM USERS WHERE review_num > {min_review_num} ORDER BY {return_type} {direction}')
    rows = cursor.fetchall()
    
    data_to_return = []
     
#     print(rows)
            
    for row in rows:
        
        data_to_return.append(
            {
            'user_id': f"{row[0]}", 
            'review_profilename': f"{row[1]}",
            'review_num': f"{row[2]}",
            'overall_mean': f"{round(row[3],3)}",
            'aroma_mean': f"{round(row[4],2)}",
            'appearance_mean': f"{round(row[5],2)}",
            'palate_mean': f"{round(row[6],2)}",
            'taste_mean': f"{round(row[7],2)}",
            'beer_abv_mean': f"{round(row[8],2)}"
            }
        )
        
    return jsonify(data_to_return)

def take_beer_abv_to_sort(arr):
    return float(arr['beer_abv'])

def take_review_num_to_sort(arr):
    return int(arr['review_num'])

def take_overall_mean_to_sort(arr):
    return float(arr['overall_mean'])

def take_aroma_mean_to_sort(arr):
    return float(arr['aroma_mean'])

def take_appearance_mean_to_sort(arr):
    return float(arr['appearance_mean'])

def take_palate_mean_to_sort(arr):
    return float(arr['palate_mean'])

def take_taste_mean_to_sort(arr):
    return float(arr['taste_mean'])

def take_overall_mean_lower_to_sort(arr):
    return float(arr['overall_mean_lower'])

@app.route('/RECOMMENDATIONS/<beer_max>/<model_name>/<user_id>/<sorting_column>/<order>/<min_review_num>')
def return_random_user_recommendations(beer_max, model_name, user_id, sorting_column, order, min_review_num):
    
    not_a_prediction = "'not_a_prediction'"
    current_model = "'" + model_name + "'"
    
    if (order=="DESC"):
        sort_reverse = True
    else:
        sort_reverse = False
        
    #------------------------------------------------------------------------------------  
    reviewed_beers = []
    
    cursor.execute(f'SELECT * FROM REVIEWS WHERE user_id={user_id} AND model={not_a_prediction} ORDER BY rating {order}')
    reviewed_rating_rows = cursor.fetchall()
    
    reviwed_beer = []
    for row in reviewed_rating_rows:
        cursor.execute(f'SELECT * FROM BEER WHERE beer_id={row[1]} AND review_num > {min_review_num}')
        reviewed_beer = cursor.fetchall()
        
        if (len(reviewed_beer) > 0):
            reviewed_beers.append(
                {
                'beer_id': f"{reviewed_beer[0][0]}", 
                'review_num': f"{reviewed_beer[0][1]}",
                'overall_mean': f"{round(reviewed_beer[0][2],2)}",
                'aroma_mean': f"{round(reviewed_beer[0][3],3)}",
                'appearance_mean': f"{round(reviewed_beer[0][4],2)}",
                'palate_mean': f"{round(reviewed_beer[0][5],2)}",
                'taste_mean': f"{round(reviewed_beer[0][6],2)}",
                'overall_mean_lower': f"{round(reviewed_beer[0][7],2)}",
                'brewery_name': f"{reviewed_beer[0][8]}",
                'beer_name': f"{reviewed_beer[0][9]}",
                'beer_style': f"{reviewed_beer[0][10]}",
                'beer_abv': f"{round(reviewed_beer[0][11],2)}",
                'rating': f"{round(row[2],3)}"
                }
            )
        
    if (sorting_column=='beer_abv'):
        reviewed_beers.sort(key=take_beer_abv_to_sort, reverse=sort_reverse)

    if (sorting_column=='review_num'):
        reviewed_beers.sort(key=take_review_num_to_sort, reverse=sort_reverse)

    if (sorting_column=='overall_mean'):
        reviewed_beers.sort(key=take_overall_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='aroma_mean'):
        reviewed_beers.sort(key=take_aroma_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='appearance_mean'):
        reviewed_beers.sort(key=take_appearance_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='palate_mean'):
        reviewed_beers.sort(key=take_palate_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='taste_mean'):
        reviewed_beers.sort(key=take_taste_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='overall_mean_lower'):
        reviewed_beers.sort(key=take_overall_mean_lower_to_sort, reverse=sort_reverse)

    reviewed_beers = reviewed_beers[:int(beer_max)]   
    #------------------------------------------------------------------------------------
    recommended_beers = []
    
    cursor.execute(f'SELECT * FROM REVIEWS WHERE user_id={user_id} AND model={current_model} ORDER BY rating {order}')
    recommended_rating_rows = cursor.fetchall()
    
    recommended_beer = []
    for row in recommended_rating_rows:
        cursor.execute(f'SELECT * FROM BEER WHERE beer_id={row[1]} AND review_num > {min_review_num}')
        recommended_beer = cursor.fetchall()
        
        if (len(recommended_beer) > 0):
            recommended_beers.append(
                {
                'beer_id': f"{recommended_beer[0][0]}", 
                'review_num': f"{recommended_beer[0][1]}",
                'overall_mean': f"{round(recommended_beer[0][2],2)}",
                'aroma_mean': f"{round(recommended_beer[0][3],3)}",
                'appearance_mean': f"{round(recommended_beer[0][4],2)}",
                'palate_mean': f"{round(recommended_beer[0][5],2)}",
                'taste_mean': f"{round(recommended_beer[0][6],2)}",
                'overall_mean_lower': f"{round(recommended_beer[0][7],2)}",
                'brewery_name': f"{recommended_beer[0][8]}",
                'beer_name': f"{recommended_beer[0][9]}",
                'beer_style': f"{recommended_beer[0][10]}",
                'beer_abv': f"{round(recommended_beer[0][11],2)}",
                'rating': f"{round(row[2],3)}"
                }
            )
        
    if (sorting_column=='beer_abv'):
        recommended_beers.sort(key=take_beer_abv_to_sort, reverse=sort_reverse)

    if (sorting_column=='review_num'):
        recommended_beers.sort(key=take_review_num_to_sort, reverse=sort_reverse)

    if (sorting_column=='overall_mean'):
        recommended_beers.sort(key=take_overall_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='aroma_mean'):
        recommended_beers.sort(key=take_aroma_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='appearance_mean'):
        recommended_beers.sort(key=take_appearance_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='palate_mean'):
        recommended_beers.sort(key=take_palate_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='taste_mean'):
        recommended_beers.sort(key=take_taste_mean_to_sort, reverse=sort_reverse)

    if (sorting_column=='overall_mean_lower'):
        recommended_beers.sort(key=take_overall_mean_lower_to_sort, reverse=sort_reverse)
        
    recommended_beers = recommended_beers[:int(beer_max)]
    #------------------------------------------------------------------------------------
    cursor.execute(f'SELECT * FROM USERS WHERE user_id={user_id}')
    user_info_row = cursor.fetchall()
    
    user_info = {
        'user_id': f"{user_info_row[0][0]}", 
        'review_profilename': f"{user_info_row[0][1]}",
        'review_num': f"{user_info_row[0][2]}",
        'overall_mean': f"{round(user_info_row[0][3],3)}",
        'aroma_mean': f"{round(user_info_row[0][4],2)}",
        'appearance_mean': f"{round(user_info_row[0][5],2)}",
        'palate_mean': f"{round(user_info_row[0][6],2)}",
        'taste_mean': f"{round(user_info_row[0][7],2)}",
        'beer_abv_mean': f"{round(user_info_row[0][8],3)}"
    }
    #------------------------------------------------------------------------------------
    
    data_to_return = {
        'user_info': user_info,
        'recommended_beers': recommended_beers,
        'reviewed_beers': reviewed_beers,
    }
        
    return jsonify(data_to_return)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2022 15:24:23] "GET /BEER_TOP/18/DESC/overall_mean_lower/20 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:24:52] "GET /USER_TOP/18/DESC/review_num/15 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:24:53] "GET /BEER_TOP/18/DESC/overall_mean_lower/20 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:24:55] "GET /RECOMMENDATIONS/17/keras_mf_estimations/0/rating/DESC/0 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:25:08] "GET /USER_TOP/18/DESC/review_num/15 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:25:09] "GET /BEER_TOP/18/DESC/overall_mean_lower/20 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:25:12] "GET /RECOMMENDATIONS/17/keras_mf_estimations/0/rating/DESC/0 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:25:13] "GET /USER_TOP/18/DESC/review_num/15 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:25:25] "GET /BEER_TOP/18/DESC/overall_mean_lower/20 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2022 15:25:26] "GET /RECOM